In [14]:
from dotenv import dotenv_values
config = dotenv_values("../.env")

In [15]:
BUCKET_SECRET_KEY = config.get('BUCKET_SECRET_KEY', None)
BUCKET_ACCESS_KEY = config.get('BUCKET_ACCESS_KEY', None)
BUCKET_ENDPOINT_URL = config.get('BUCKET_ENDPOINT_URL', None)
BUCKET_NAME = config.get('BUCKET_NAME', None)

In [16]:
BUCKET_NAME

'storage'

In [21]:
from datetime import datetime, timezone
from urllib.parse import quote
from typing import Optional
from dotenv import dotenv_values

class S3Handler:
    def __init__(self):
        # Load credentials from .env file
        config = dotenv_values("../.env")
        
        self.endpoint_url = config.get('BUCKET_ENDPOINT_URL')
        self.bucket_name = config.get('BUCKET_NAME')
        self.access_key = config.get('BUCKET_ACCESS_KEY')
        self.secret_key = config.get('BUCKET_SECRET_KEY')
        self.region = 'us-east-1'
        
        from botocore.client import Config as BotocoreConfig
        import boto3
        
        botocore_config = BotocoreConfig(
            # signature_version='s3v4',
            signature_version='s3',
            s3={
                "addressing_style": "path",
                'payload_signing_enabled': True     
            }
        )


        self.bucket_name = BUCKET_NAME
        self.client = boto3.client(
            's3',
            endpoint_url=self.endpoint_url,
            aws_access_key_id=self.access_key,
            region_name='us-east-1', 
            aws_secret_access_key=self.secret_key,
            use_ssl=False,
            config=botocore_config
        )
    
    def generate_presigned_url(self, s3_key: str, expiration: int = 3600) -> Optional[str]:
        """
        Generate presigned URL using boto3
        """
        try:
            url = self.client.generate_presigned_url(
                'get_object',
                Params={
                    'Bucket': self.bucket_name,
                    'Key': s3_key
                },
                ExpiresIn=expiration
            )
            return url
        except Exception as e:
            print(f"Error generating presigned URL: {e}")
            return None
    
    def upload_file(self, file_content: bytes, content_type: str, s3_key: str) -> bool:
        try:
            self.client.put_object(
                Bucket=self.bucket_name,
                Key=s3_key,
                Body=file_content,
                ContentType=content_type
            )
            print(f"Successfully uploaded {s3_key}")
            return True
        except Exception as e:
            print(f"Upload error: {e}")
            return False
    
    def bucket_exists(self) -> bool:
        try:
            self.client.head_bucket(Bucket=self.bucket_name)
            return True
        except Exception:
            return False

In [18]:
s3 = S3Handler()

In [19]:
r = s3.generate_presigned_url('content/sttus-.png')

In [20]:
r

'http://localhost:400/storage/content/sttus-.png?AWSAccessKeyId=f5EEOB1EFpTZyC8i3PCE&Signature=Ir5HIK7OTvClyZb4zstypMs1J%2Fg%3D&Expires=1769626635'

In [24]:
from hashlib import sha256


with open('status.png', 'rb') as df:
    h = sha256()
    h.update(df.read())
    
    df.seek(0)
    s3.upload_file(
        file_content=df.read(),
        content_type='image/png',
        s3_key=f'content/{h.hexdigest()}.png'
    )
    ref = s3.generate_presigned_url(s3_key=f'content/{h.hexdigest()}.png')
    print(ref)

Successfully uploaded content/a73b219a3d203d34efcb0891c3ec2c447e7bac16fded8a98a1353ec1facf44f4.png
http://localhost:400/storage/content/a73b219a3d203d34efcb0891c3ec2c447e7bac16fded8a98a1353ec1facf44f4.png?AWSAccessKeyId=f5EEOB1EFpTZyC8i3PCE&Signature=23DT2BSU3nVns2D3wXzHasTvPqE%3D&Expires=1769626953
